In [1]:
import findspark
findspark.init('D:\spark-2.3.3-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
print('modules imported')

modules imported


In [2]:
spark= SparkSession.builder.appName('spark_study2').getOrCreate()
print('app name2')

app name2


In [4]:
#handling null values in dataset
null_rdd=spark.read.csv('D:\python coding\pyspark_tutorial\ContainsNull.csv',header=True,inferSchema=True)
null_rdd.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [11]:
#na func to drop rows with null values
null_rdd.na.drop().show() #rows having atleast a null value is dropped
null_rdd.na.drop(thresh=2).show() #rows having nulls greater than 2 are dropped


#drop rows having nulls using how parameter
null_rdd.na.drop(how='any').show() #records having atleast a null wull be dropped
null_rdd.na.drop(how='all').show() #record having all nulls will be dropped

#dropping null values on basis of a column
null_rdd.na.drop(subset=['Sales']).show()



+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [16]:
#filling null values into dataset
null_rdd.na.fill('NA').show() #spark automatically detects if a column is string or numeric
null_rdd.na.fill(0).show() 

#filling on basis of column name
null_rdd.na.fill('No Name',subset=['Name']).show()

#filling multiple column values basis of datatypes
null_rdd.na.fill({'Name': 'No Name', 'Sales': 0}).show()


+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2|   NA| null|
|emp3|   NA|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| null|  0.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John| null|
|emp2|No Name| null|
|emp3|No Name|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John|  0.0|
|emp2|No Name|  0.0|
|emp3|No Name|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+



In [27]:
#filling numeric column values with the mean or average value of that particular column
from pyspark.sql.functions import mean
mean_val=null_rdd.select(mean(null_rdd.Sales)).collect() 
mean_val  #mean_val is a list 
mean_sales=mean_val[0][0]

#now using men_sales value to fill the nulls in sales column
null_rdd.na.fill(mean_sales,subset=['Sales']).show()


+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [35]:
#working with dates in pyspark
date_rdd=spark.read.csv('D:\\python coding\\pyspark_tutorial\\appl_stock.csv',header=True,inferSchema=True)
#date_rdd.show()

date_rdd.select('Date','Open').show()

#date functions
from pyspark.sql.functions import (dayofmonth,hour,dayofyear,month,year,weekofyear,format_number,date_format)

#to see date part from date format(yyyy-mm-dd)
date_rdd.select(dayofmonth(date_rdd.Date)).show()

+-------------------+------------------+
|               Date|              Open|
+-------------------+------------------+
|2010-01-04 00:00:00|        213.429998|
|2010-01-05 00:00:00|        214.599998|
|2010-01-06 00:00:00|        214.379993|
|2010-01-07 00:00:00|            211.75|
|2010-01-08 00:00:00|        210.299994|
|2010-01-11 00:00:00|212.79999700000002|
|2010-01-12 00:00:00|209.18999499999998|
|2010-01-13 00:00:00|        207.870005|
|2010-01-14 00:00:00|210.11000299999998|
|2010-01-15 00:00:00|210.92999500000002|
|2010-01-19 00:00:00|        208.330002|
|2010-01-20 00:00:00|        214.910006|
|2010-01-21 00:00:00|        212.079994|
|2010-01-22 00:00:00|206.78000600000001|
|2010-01-25 00:00:00|202.51000200000001|
|2010-01-26 00:00:00|205.95000100000001|
|2010-01-27 00:00:00|        206.849995|
|2010-01-28 00:00:00|        204.930004|
|2010-01-29 00:00:00|        201.079996|
|2010-02-01 00:00:00|192.36999699999998|
+-------------------+------------------+
only showing top

In [47]:
#calculating avg closing price yearwise
#using withColumn we have created a new column which will hold year part
#similar to oracle to_char(date,'yyyy') 
new_rdd=date_rdd.withColumn("Year",year(date_rdd.Date))
new_rdd.select('Date','Open','Year').show()
#in next step we will perform group by operation
new_rdd.groupBy('Year').mean().select(['Year','avg(Open)']).show()
#avergae open price per year
#performing display changes for decimal value
new=new_rdd.withColumnRenamed('avg(open)','Average open rate')
new.select('Date','Average open rate'),show()
#new.select(['Year',format_number('Average open rate',2).alias('Avg rate')]).show()

+-------------------+------------------+----+
|               Date|              Open|Year|
+-------------------+------------------+----+
|2010-01-04 00:00:00|        213.429998|2010|
|2010-01-05 00:00:00|        214.599998|2010|
|2010-01-06 00:00:00|        214.379993|2010|
|2010-01-07 00:00:00|            211.75|2010|
|2010-01-08 00:00:00|        210.299994|2010|
|2010-01-11 00:00:00|212.79999700000002|2010|
|2010-01-12 00:00:00|209.18999499999998|2010|
|2010-01-13 00:00:00|        207.870005|2010|
|2010-01-14 00:00:00|210.11000299999998|2010|
|2010-01-15 00:00:00|210.92999500000002|2010|
|2010-01-19 00:00:00|        208.330002|2010|
|2010-01-20 00:00:00|        214.910006|2010|
|2010-01-21 00:00:00|        212.079994|2010|
|2010-01-22 00:00:00|206.78000600000001|2010|
|2010-01-25 00:00:00|202.51000200000001|2010|
|2010-01-26 00:00:00|205.95000100000001|2010|
|2010-01-27 00:00:00|        206.849995|2010|
|2010-01-28 00:00:00|        204.930004|2010|
|2010-01-29 00:00:00|        201.0

AnalysisException: "cannot resolve '`Average open rate`' given input columns: [High, Open, Volume, Date, Close, Adj Close, Year, Low];;\n'Project [Date#1120, 'Average open rate]\n+- Project [Date#1120, Open#1121, High#1122, Low#1123, Close#1124, Volume#1125, Adj Close#1126, year(cast(Date#1120 as date)) AS Year#1828]\n   +- Relation[Date#1120,Open#1121,High#1122,Low#1123,Close#1124,Volume#1125,Adj Close#1126] csv\n"